In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
from PIL import Image  

def distance_matrix(points1, points2):
    if points2.size == 0:
        return np.array([])

    diff = points1[:, np.newaxis, :] - points2[np.newaxis, :, :]
    return np.sqrt(np.sum(diff**2, axis=2))
 

def showImage(image_array):  
    if image_array.shape[0] > 1000 or image_array.shape[1] > 1000:  
        image_array = image_array[::2, ::2] 

    plt.imshow(image_array)  
    plt.axis('off')  
    plt.show()   

class DBSCAN_for_segmentation:  
    
    def __init__(self, r_picture, r_color):  
        self.image = []  
        self.image_copy = []  
        self.r_picture = r_picture  
        self.r_color = r_color  
        self.clusters = []  
    
    def add_to_clusters(self, x, y, which_cluster):
        if len(self.clusters) <= which_cluster:
            self.clusters.append([])
    
        stack = [(x, y)]
        
        while stack:
            curr_x, curr_y = stack.pop()
            
            if self.image_copy[curr_x][curr_y][5] == 1:  
                continue
            
            self.clusters[which_cluster].append(self.image_copy[curr_x][curr_y])
            self.image_copy[curr_x][curr_y][5] = 1 
    
            # Define the search area
            i_first = max(0, curr_x - self.r_picture)  
            j_first = max(0, curr_y - self.r_picture)  
            i_second = min(len(self.image_copy), curr_x + self.r_picture + 1)  
            j_second = min(len(self.image_copy[0]), curr_y + self.r_picture + 1)  
    
            search_area = [(i, j) for i in range(i_first, i_second) for j in range(j_first, j_second) if self.image_copy[i][j][5] == 0]
    
            if not search_area:
                continue
            
            unvisited_points = np.array(search_area)
            cluster_color = np.array([self.image_copy[curr_x][curr_y][2:5]])  # Current pixel color
    
            distances = distance_matrix(cluster_color, np.array([self.image_copy[i][j] for (i, j) in unvisited_points])[:, 2:5])  
            
            # Find neighbors within the color distance
            neighbors = np.where(distances[0] <= self.r_color)[0]
            
            for neighbor_index in neighbors:
                i, j = unvisited_points[neighbor_index]
                stack.append((i, j))
    
    
    def fit(self, image):  
        if len(self.clusters) > 1:
            self.clusters = []
        self.image = image  
        self.image_copy = []  
        
        for i in range(len(image)):  
            temp = []  
            for j in range(len(image[i])):  
                temp.append([i, j, *image[i][j][:3], 0])
            self.image_copy.append(temp)  
        print("clustering started")
        which_cluster = 0  
        for i in range(len(self.image_copy)):  
            for j in range(len(self.image_copy[i])):  
                if self.image_copy[i][j][5] == 0:
                    self.add_to_clusters(i, j, which_cluster)  
                    which_cluster += 1  
        
        finall_image = np.zeros((len(self.image_copy), len(self.image_copy[0]), 3), dtype=np.uint8)    

        cluster_colors = {i: [np.random.randint(0, 255) for _ in range(3)] for i in range(len(self.clusters))}  
   
        for i, cluster in enumerate(self.clusters):  
            for pixel in cluster:  
                x, y = pixel[0], pixel[1]  
                finall_image[x, y] = cluster_colors[i]
                  
        if finall_image.shape[0] > 1000 or finall_image.shape[1] > 1000:  
            finall_image = finall_image[::2, ::2] 
        print("Number of clusters formed: ", len(self.clusters))  
        #showImage(finall_image)  # Show the final segmented image  
        return finall_image

if __name__ == "__main__":  
    image_path = 'lena.png'  
    image = Image.open(image_path)  
    image = np.array(image)  
    
    db = DBSCAN_for_segmentation(r_color=5, r_picture=3)  
    lena = db.fit(image)
    fig, axs = plt.subplots(2, 2, figsize=(8, 5)) 

    axs[0][0].imshow(image)
    axs[0][0].axis('off') 
    axs[0][0].set_title('lena before')

    axs[0][1].imshow(lena)
    axs[0][1].axis('off') 
    axs[0][1].set_title('lena - after')

    image_path = 'peppers.tif'  
    image = Image.open(image_path)  
    image = np.array(image)  
     
    veg = db.fit(image)
    
    axs[1][0].imshow(image)
    axs[1][0].axis('off') 
    axs[1][0].set_title('vegetable - before')
    axs[1][1].imshow(veg)
    axs[1][1].axis('off') 
    axs[1][1].set_title('vegetable - after')

    plt.tight_layout()
    plt.show()

: 